In [38]:
%pip install --upgrade setuptools wheel 

Note: you may need to restart the kernel to use updated packages.


In [39]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [40]:
%pip install wheel pymysql mysql-connector-python sqlalchemy schedule

Note: you may need to restart the kernel to use updated packages.


In [41]:
%pip install git+https://github.com/heebp/pykrx.git


  Cloning https://github.com/heebp/pykrx.git to c:\users\student\appdata\local\temp\pip-req-build-qjahyi8t
  Resolved https://github.com/heebp/pykrx.git to commit b0ffe3651dfb7037d5494d03b16a04622aec5a6a
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
Note: you may need to restart the kernel to use updated packages.


  Running command git clone --filter=blob:none --quiet https://github.com/heebp/pykrx.git 'C:\Users\student\AppData\Local\Temp\pip-req-build-qjahyi8t'


In [42]:
from pykrx import stock
from pykrx import bond
import os
from dotenv import load_dotenv
import pandas as pd
import datetime
from sqlalchemy import create_engine, MetaData, Table
from sqlalchemy.orm import sessionmaker

load_dotenv()

DB_HOST = os.getenv("DB_HOST")
DB_USERNAME = os.getenv("DB_USERNAME")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_SCHEME = os.getenv("DB_SCHEME")

db_username = DB_USERNAME
db_password = DB_PASSWORD
db_host = DB_HOST
db_name = DB_SCHEME

engine = create_engine(f'mysql+pymysql://{db_username}:{db_password}@{db_host}/{db_name}')

In [43]:
def get_stock_history_columns(table_name):
  metadata = MetaData()
  table = Table(table_name, metadata, autoload_with=engine)
  db_columns = [column.name for column in table.columns]
  return db_columns

In [44]:
def get_price_data(at_date):
  df1 = stock.get_market_ohlcv(at_date, market="KOSPI")
  df2 = stock.get_market_ohlcv(at_date, market="KOSDAQ")

  result = pd.concat([df1, df2], axis=0)

  return result

In [45]:
def get_stock_id_by_short_code():
  Session = sessionmaker(bind=engine)
  session = Session()

  metadata = MetaData()
  stock_table = Table('STOCK', metadata, autoload_with=engine)

  # stock 테이블에서 short_code와 stock_id 조회
  query = session.query(stock_table.c.SHORT_CODE, stock_table.c.STOCK_ID).all()

  session.close()
  
  # short_code와 stock_id를 매핑한 딕셔너리 생성
  return {row.SHORT_CODE: row.STOCK_ID for row in query}


In [46]:
def convert_columns(data):
  data = data.reset_index()

  column_mapping = {
    '티커': 'SHORT_CODE',
    '시가': 'OPEN_PRICE',
    '종가': 'CLOSED_PRICE',
    '고가': 'HIGH_PRICE',
    '저가': 'LOW_PRICE',
  }

  data.rename(columns=column_mapping, inplace=True)
  short_code_to_stock_id_map = get_stock_id_by_short_code()
  data['STOCK_ID'] = data['SHORT_CODE'].map(short_code_to_stock_id_map)
  data['STOCK_ID'] = data['STOCK_ID'].astype(int)
  return data

In [47]:
def fetch_KOSPI_KOSDAQ_price_data():

  price = get_price_data(datetime.datetime.now().strftime("%Y%m%d"))

  price = convert_columns(price)

  return price[['STOCK_ID','SHORT_CODE','CLOSED_PRICE','OPEN_PRICE','HIGH_PRICE','LOW_PRICE']]

In [48]:
def fetch_and_save_data(time):
    # 주식 데이터를 가져오는 함수 호출
    data = fetch_KOSPI_KOSDAQ_price_data()
    print(data)
    data['STOCK_HISTORY_ID'] = time
    data.to_sql(name='STOCK_HISTORY', con=engine, if_exists='append', index=False)
    print("Data saved at:", datetime.datetime.now())


In [49]:

# 주식 개장 시간인지 확인하는 함수
def is_market_open():
    current_time = datetime.datetime.now().time()
    market_open_time = datetime.time(9, 0)  # 09:00 AM
    market_close_time = datetime.time(15, 30)  # 03:30 PM
    return market_open_time <= current_time <= market_close_time


In [52]:
import datetime

now = datetime.datetime.now()

fetch_and_save_data(now.strftime("%Y%m%d%H%M%S"))        


      STOCK_ID SHORT_CODE  CLOSED_PRICE
0        28656     095570          4755
1        27529     006840         12990
2        27890     027410          3535
3        29326     282330        110600
4        28891     138930          9210
...        ...        ...           ...
2708     27845     024060         21200
2709     27629     010240          4880
2710     29014     189980          1794
2711     28041     037440          5150
2712     29198     238490          4690

[2713 rows x 3 columns]


IntegrityError: (pymysql.err.IntegrityError) (1062, "Duplicate entry '2147483647-28656' for key 'STOCK_HISTORY.PRIMARY'")
[SQL: INSERT INTO `STOCK_HISTORY` (`STOCK_ID`, `SHORT_CODE`, `CLOSED_PRICE`, `STOCK_HISTORY_ID`) VALUES (%(STOCK_ID)s, %(SHORT_CODE)s, %(CLOSED_PRICE)s, %(STOCK_HISTORY_ID)s)]
[parameters: [{'STOCK_ID': 28656, 'SHORT_CODE': '095570', 'CLOSED_PRICE': 4755, 'STOCK_HISTORY_ID': '20241007125915'}, {'STOCK_ID': 27529, 'SHORT_CODE': '006840', 'CLOSED_PRICE': 12990, 'STOCK_HISTORY_ID': '20241007125915'}, {'STOCK_ID': 27890, 'SHORT_CODE': '027410', 'CLOSED_PRICE': 3535, 'STOCK_HISTORY_ID': '20241007125915'}, {'STOCK_ID': 29326, 'SHORT_CODE': '282330', 'CLOSED_PRICE': 110600, 'STOCK_HISTORY_ID': '20241007125915'}, {'STOCK_ID': 28891, 'SHORT_CODE': '138930', 'CLOSED_PRICE': 9210, 'STOCK_HISTORY_ID': '20241007125915'}, {'STOCK_ID': 27266, 'SHORT_CODE': '001460', 'CLOSED_PRICE': 32500, 'STOCK_HISTORY_ID': '20241007125915'}, {'STOCK_ID': 27267, 'SHORT_CODE': '001465', 'CLOSED_PR

In [50]:
import schedule
import time
import datetime

def stock_market_schedule():
    while True:
        now = datetime.datetime.now()

        # 주식 개장 시간 (9:00 ~ 15:30) 체크
        if now.hour >= 9 and (now.hour < 15 or (now.hour == 15 and now.minute <= 30)):
            
            # 현재 시간이 10분 단위이고, 초가 00초일 때 실행
            if now.minute % 10 == 0 and now.second == 0:
                fetch_and_save_data(now.strftime("%Y%m%d%H%M%S"))        
        # 1초 대기 후 다시 체크
        time.sleep(1)

# 스케줄 시작
stock_market_schedule()

      STOCK_ID SHORT_CODE  CLOSED_PRICE
0        28656     095570          4745
1        27529     006840         12990
2        27890     027410          3535
3        29326     282330        110700
4        28891     138930          9210
...        ...        ...           ...
2708     27845     024060         21000
2709     27629     010240          4880
2710     29014     189980          1795
2711     28041     037440          5120
2712     29198     238490          4685

[2713 rows x 3 columns]


IntegrityError: (pymysql.err.IntegrityError) (1062, "Duplicate entry '2147483647-28656' for key 'STOCK_HISTORY.PRIMARY'")
[SQL: INSERT INTO `STOCK_HISTORY` (`STOCK_ID`, `SHORT_CODE`, `CLOSED_PRICE`, `STOCK_HISTORY_ID`) VALUES (%(STOCK_ID)s, %(SHORT_CODE)s, %(CLOSED_PRICE)s, %(STOCK_HISTORY_ID)s)]
[parameters: [{'STOCK_ID': 28656, 'SHORT_CODE': '095570', 'CLOSED_PRICE': 4745, 'STOCK_HISTORY_ID': '20241007125000'}, {'STOCK_ID': 27529, 'SHORT_CODE': '006840', 'CLOSED_PRICE': 12990, 'STOCK_HISTORY_ID': '20241007125000'}, {'STOCK_ID': 27890, 'SHORT_CODE': '027410', 'CLOSED_PRICE': 3535, 'STOCK_HISTORY_ID': '20241007125000'}, {'STOCK_ID': 29326, 'SHORT_CODE': '282330', 'CLOSED_PRICE': 110700, 'STOCK_HISTORY_ID': '20241007125000'}, {'STOCK_ID': 28891, 'SHORT_CODE': '138930', 'CLOSED_PRICE': 9210, 'STOCK_HISTORY_ID': '20241007125000'}, {'STOCK_ID': 27266, 'SHORT_CODE': '001460', 'CLOSED_PRICE': 32500, 'STOCK_HISTORY_ID': '20241007125000'}, {'STOCK_ID': 27267, 'SHORT_CODE': '001465', 'CLOSED_PR